## Configuration

In [1]:
### Configuration
# Import dependencies
import os
from collections import Counter
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
from summerplaylist import *

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

## Log In

In [2]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Read in current song candidates

In [3]:
# Get playlist IDs
playlistID1 = getPlaylistID(credentials, '2023')
playlistID2 = getPlaylistID(credentials, '2023 Round 2')

# Read in CurrentYearSongCandidates playlist
playlistTracks1 = fetchTracksFromPlaylist(credentials, playlistID1)
playlistTracks2 = fetchTracksFromPlaylist(credentials, playlistID2)
# Merge dfs
allNewTracks = pd.concat([playlistTracks1, playlistTracks2], axis=0, ignore_index=True)
allNewTracks

## Read in old playlists

In [5]:
# Read in all old playlists
pl2018 = getPlaylistID(credentials, "Summer 2018")
plt2018 = fetchTracksFromPlaylist(credentials, pl2018)
pl2019 = getPlaylistID(credentials, "Summer 2019")
plt2019 = fetchTracksFromPlaylist(credentials, pl2019)
pl2020 = getPlaylistID(credentials, "Endless Summer 2020")
plt2020 = fetchTracksFromPlaylist(credentials, pl2020)
pl2021 = getPlaylistID(credentials, "Endless Summer 2021")
plt2021 = fetchTracksFromPlaylist(credentials, pl2021)
pl2022 = getPlaylistID(credentials, "Endless Summer 2022")
plt2022 = fetchTracksFromPlaylist(credentials, pl2022)
# Merge dfs
allOldTracks = pd.concat([plt2018, plt2019, plt2020, plt2021, plt2022], axis=0, ignore_index=True)

## Filter out non-qualifiers

In [36]:
# Filter out any songs that have already appeared in old playlists
# Based on songURI alone
newFiltered = allNewTracks.loc[~allNewTracks['songURI'].isin(allOldTracks['songURI'])]
print(len(newFiltered))
# Based on artist ID + trackName, which is more exact
allNewTracks['combined'] = allNewTracks['artistID']+allNewTracks['trackName']
allOldTracks['combined'] = allOldTracks['artistID']+allOldTracks['trackName']
newFiltered = newFiltered.loc[~newFiltered['combined'].isin(allOldTracks['combined'])]
print(len(newFiltered))

# Filter out any songs older than 5 years
dateMin = datetime((datetime.today() - relativedelta(years=5)).year, 3, 17)
newFiltered['release_date'] = pd.to_datetime(newFiltered['release_date'])
newFiltered = newFiltered.loc[newFiltered['release_date']>dateMin].reset_index(drop=True)
print(len(newFiltered))

# Sort by frequency of artist, artist
# Get frequency of artists in previous summers
frequency = pd.DataFrame(sorted(Counter(allOldTracks['artistID']).items(), key=lambda x:x[1], reverse=True))
frequency.columns = ['artistID', 'artistAppearances']

# Merge with new 
newFiltered = newFiltered.merge(frequency, how='left', on='artistID')
newFiltered['artistAppearances'] = newFiltered['artistAppearances'].fillna(0)

# Sort
newFiltered = newFiltered.sort_values(by=['artistAppearances', 'artistID', 'release_date'], ascending=False, ignore_index=True)

# Drop duplicates
newFiltered = newFiltered.drop_duplicates(subset='songURI', ignore_index=True)
print(len(newFiltered))

704
698
686
478


## Create new playlist

In [ ]:
# Warning: This will make a new playlist
playlistNew = getPlaylistID('2023 Cull')
# Add songs to the playlist
addToPlaylist(credentials, newFiltered['songURI'], playlistNew)